<a href="https://colab.research.google.com/github/JacquesBLR/Causal_Inference/blob/main/Estimation_de_l%E2%80%99%C3%89lasticit%C3%A9_Prix_%E2%80%93_Approche_par_Variables_Instrumentales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contexte

Une entreprise de distribution veut **quantifier l’effet causal du prix sur les ventes d’un produit
(l’élasticité prix)**.  
En marketing, la question “de combien une baisse de prix va-t-elle augmenter les
ventes ?” est cruciale pour la tarification.  
Cependant, **mesurer cet effet est difficile car en données
observationnelles, le prix et la demande sont corrélés par de multiples mécanismes**.  
Par exemple, si on observe une corrélation positive entre prix et ventes sur l’historique, ce n’est pas forcément parce
qu’un **prix élevé stimule les ventes (peu plausible !)**, mais peut-être parce que **l’entreprise augmente les
prix lorsque la demande est forte** (causalité inverse) ou que des facteurs confondants (saison,
promotion, qualité du produit) influencent les deux.  
Dans ce cas, **un A/B test direct est souvent impossible** : on ne peut pas facilement proposer
simultanément deux prix différents sur le même marché sans biais (sauf à faire des tests
géographiques contrôlés, mais supposons que ce n’est pas fait ici).  
**On recourt alors aux variables
instrumentales (VI) pour estimer l’impact du prix de manière causale**.  
**Idée d’une variable instrumentale : trouver un facteur extérieur qui influence le prix, mais
n’influence pas directement la demande, de sorte qu’il crée une variation “quasi-expérimentale” du prix**.  
On utilisera cette variation exogène pour estimer l’effet sur les ventes.
  
  **Classiquement en économie, le coût de production/matière première est un bon candidat : si le coût augmente, l’entreprise peut répercuter en partie sur le prix.**  
**Le coût est corrélé au prix, mais n’affecte pas la demande des consommateurs** (ceux-ci ne connaissent pas le coût interne). Ici, on supposera que les
fluctuations de coût sont indépendantes de la demande des clients (pas de corrélation avec les
préférences ou la saisonnalité par ex.).  
En pratique, on dispose d’un historique (ou panel) avec par exemple :  
colonnes *semaine* , *prix_unitaire* , *ventes_unites* , et *cout_unitaire* du produit (ainsi que d’autres variables de contrôle éventuelles comme des tendances, etc.).  
L’objectif est d’**estimer l’élasticité (la variation % des
ventes pour 1% de variation du prix)** ou du moins la pente de la relation causal ventes-prix. On va
comparer :
- Régression OLS naïve ventes ~ prix : biaisée si le prix est **endogène** (corrélé à l’erreur).
- Régression IV (2SLS) ventes ~ prix en utilisant cout comme instrument du prix : on espère corriger
le biais et obtenir une estimation non entachée par la **causalité inverse** ou les confounders non observés

# Données

Nous utiliserons un jeu de données de vente d'un produit sur N périodes (jours,
semaines…). On doit avoir suffisamment de variation en prix et coût.  
Les données simulées respectent le modèle :
Générer des chocs de demande aléatoires *u* qui influent sur les ventes.  
Générer des coûts aléatoires *cout* indépendants de *u* .  
**Modèle de pricing :** **prix = baseline + α * cout + β * u + ...**  
(l’entreprise fixe le prix en fonction du coût et peut-être un peu en fonction de la demande anticipée u , ce qui cause
l’endogénéité en corrélant prix et u).  
**Modèle de demande : ventes = gamma - δ * prix + γ * u + ...**   
(les consommateurs achètent moins si le prix est haut, et u affecte aussi les ventes – par ex. *u* représente un
engouement exogène).  
Dans ce modèle, **le prix est corrélé avec u (car l’entreprise ajuste en partie sur u), et u influence aussi les ventes.  
Donc la corrélation prix-ventes sera biaisée**.  
En revanche, le coût n’affecte pas directement les
ventes (seul effet est via prix), donc le coût peut servir d’instrument valide pour le prix (corrélé au prix,
indépendant de u par construction).


# Méthode et analyse
**1. Analyse naïve (OLS)** :  
On estime la régression simple ventes = a + b * prix + erreur par
OLS.  
En Python avec statsmodels ou en R avec lm.  
On observe le coefficient $b_{\text{OLS}}$.  
On le compare à la vraie élasticité si on la connaît (dans une simulation on la connaît par construction).  
Souvent, $b_{\text{OLS}}$ sera biaisé vers 0 ou de signe opposé si l’endogénéité est forte.  
Par exemple,
on pourrait obtenir une pente faiblement négative alors que la vraie relation est fortement négative.
Dans certains cas extrêmes, OLS pourrait même donner un coefficient positif si le biais est très fort (ce
qui aurait signifié plus le prix est haut, plus on vend, paradoxal).  

**2. Estimation par variable instrumentale (2SLS)** :   
On applique la méthode en deux étapes :  
- Première étape: régression du prix sur l’instrument (et éventuellement d’autres contrôles).  
Ici **prix = a1 + c * cout + ...** . On obtient le prix “prédit” influencé seulement par la partie
exogène (le coût).
- Deuxième étape: régression des ventes sur le prix prédit :  
. **ventes = a2 + b_{\text{IV}} x prix_prédit + ...**. Le coefficient $b_{\text{IV}}$ est l’estimation de l’effet causal du prix sur les
ventes.

En Python, il n’y a pas de package standard de VI dans la bibliothèque standard, mais on peut
facilement coder ces deux étapes ou utiliser linearmodels si installé. En R, on utilisera le package
AER ( ivreg ) ou la fonction TSLS du package sem pour aller plus vite.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
np.random.seed(7)
N = 200
# Variables exogènes
demand_shock = np.random.normal(0, 1, N) # choc de demande (non observé en pratique)
cost = np.random.normal(0, 1, N) # coût matière
# Générer prix en fonction de coût et (malheureusement) partiellement du choc de demande
price = 100 + 3 * cost + 5 * demand_shock
# l'entreprise répercute coûts et anticipe demande
# Générer ventes en fonction du prix et du choc de demande
sales = 1000 - 10 * price + 30 * demand_shock + np.random.normal(0, 10, N)
data = pd.DataFrame({'prix': price, 'ventes': sales, 'cout': cost})
# Régression OLS naïve: ventes ~ prix
ols_model = sm.OLS(data['ventes'], sm.add_constant(data[['prix']])).fit()
print("OLS Coefficient prix = {:.2f}, p-value = {:.3f}".format(ols_model.params['prix'], ols_model.pvalues['prix']))
# Régression IV en deux étapes:
# 1ère étape: prix ~ cout
fs_model = sm.OLS(data['prix'], sm.add_constant(data[['cout']])).fit()
data['prix_chap'] = fs_model.predict(sm.add_constant(data[['cout']]))
# 2ème étape: ventes ~ prix_chap
ss_model = sm.OLS(data['ventes'], sm.add_constant(data[['prix_chap']])).fit()
print("IV Coefficient prix = {:.2f}, p-value = {:.3f}".format(ss_model.params['prix_chap'], ss_model.pvalues['prix_chap']))

OLS Coefficient prix = -5.47, p-value = 0.000
IV Coefficient prix = -10.18, p-value = 0.000


Ici, l’OLS naïf estime qu’augmenter le prix d’1 unité fait baisser les ventes d’environ 5.5 unités (b = -5.47).  
La méthode par VI estime une pente d’environ -10.2, proche de la véritable valeur -10 utilisée dans la
simulation.  
La différence est massive : **OLS sous-estimait de moitié l’effet du prix**.

 Pourquoi ? Parce que
dans les données, chaque fois que la demande latente était forte (demand_shock élevé), le prix avait
tendance à être relevé et les ventes finissaient de toute façon plus élevées, ce qui a atténué
artificiellement la pente observée.  
L’instrument “coût”, lui, fournit une variation de prix indépendante de
la demande, révélant l’effet réel : **quand le coût (et donc le prix) monte, la demande réagit fortement à
la baisse**.

**3. Vérifications et interprétation :**   
Vérifier la pertinence de l’instrument : on peut regarder la régression de première étape prix ~ cout , **le coefficient de cout
doit être clairement non nul (ici ~3.0) et significatif**.  
**Un test F de l'instrument > 10 est un critère classique de force de l’instrument**.  
Dans notre script, on peut imprimer fs_model.rsquared ou le F-stat.  
Si l’instrument est faible (peu corrélé au prix), l’estimation IV sera peu fiable (erreurs-types grandes). Ici ce n’est pas le cas.

**Hypothèse d’exclusion :**  
Peut-on raisonnablement penser que les fluctuations de coût
n’affectent les ventes qu’à travers le prix ?  
Dans un contexte réel, il faut y réfléchir : si une hausse de coût
entraîne aussi une baisse de qualité du produit ou moins de pub (pour compenser marge), alors cout
influencerait les ventes autrement que via le prix – l’instrument ne serait plus valide. On supppose ici
que ce n’est pas le cas

**Interprétation du coefficient** :  
$b_{\text{IV}} \approx -10.2$ **signifie “toute chose égale par ailleurs, une
hausse de 1€ du prix entraîne une baisse d’environ 10 unités vendues” sur la période considérée**.  
On peut
convertir en élasticité : élasticité $= b * \frac{\bar{prix}}{\bar{ventes}}$.  
 Si par ex. prix moyen ~100€, ventes moy ~1000 unités, alors élasticité ~ $-10 * (100/1000) = -1.0$, soit une élasticité de -1 (chaque
+1% prix => -1% ventes).

**4. Utilisation du résultat** :  
Avec une telle estimation, l’entreprise peut simuler des scenarios de
tarification.  
Par ex, b_IV = -10 signifie baisser le prix de 5€ augmenterait les ventes de ~50 unités.  
Mais attention, cela suppose rester dans la plage de données observée ; une extrapolation trop loin peut être
risquée (élasticité non linéaire, segments de clients saturés, etc.).  
On discute aussi de la confiance : un intervalle de confiance peut être calculé pour b_IV (via summary(iv) on l’a).  
Si l’intervalle exclut 0
largement, on est confiant qu’il y a bien une relation négative significative.

# Discussion : hypothèses, limites, interprétation

Ce cas démontre la puissance des variables instrumentales pour résoudre un problème d’endogénéité classique en marketing (prix endogène) .  
Toutefois, la validité repose sur des hypothèses fortes :
- Instrument pertinent : ici, le coût doit être suffisamment corrélé au prix. Si l’entreprise ne
faisait pas varier ses prix en fonction du coût (par ex. prix fixés indépendamment des coûts à
court terme), alors le coût ne serait pas un bon instrument (faible corrélation => estimateur IV
peu fiable, grand écart-type). On a supposé que ce n’est pas le cas.
- Exclusion (exogénéité) : le coût ne doit pas affecter directement la demande. Dans la réalité,
cela peut être contesté : si une hausse de coût réduit la qualité du produit (fabrication moins chère) ou implique moins de promotion, la demande peut baisser pour ces raisons-là aussi.  
Il faut choisir un instrument où l’on est convaincu que seul le prix est le canal d’influence.  
Souvent, on cherche des “expériences naturelles” : par ex. la météo impacte les coûts
d’approvisionnement d’un ingrédient, ce qui affecte le prix du produit fini, mais la météo
n’affecte pas la demande des consommateurs pour ce produit autrement.  
Ce n’est pas trivial à trouver !  
Les marketeurs utilisent rarement des VI car c’est difficile d’en trouver de solides dans un contexte business . Mais quand on y parvient, c’est très puissant.

Dans notre simulation, l’instrument était parfait car on l’a fabriqué ainsi.

 Dans la pratique, il faut
toujours justifier qualitativement qu’un instrument est valide, et éventuellement tester ses implications
(tests de sur-identification si on a plusieurs instruments, etc.). Papies et al. (2023) notent que chaque
situation requiert une argumentation au cas par cas, il n’y a pas de solution miracle toute faite

**Limites additionnelles :** L’estimation IV donne une pente moyenne sur la plage de variation induite par
l’instrument.  
Si la relation n’est pas linéaire, ou si l’effet du prix diffère selon les segments de demande,
on obtient une sorte d’**effet local moyen (LATE)**.  
Par exemple, peut-être que l’entreprise ne varie ses
prix que dans certaines conditions (pénuries ou soldes) – l’effet mesuré correspond à ces situations.  
On doit rester conscient du contexte.  
Malgré tout, l’outil VI est extrêmement utile lorsque les autres méthodes font défaut. Il a été utilisé dans
de nombreux travaux marketing (par ex. utiliser le déploiement aléatoire de certaines
fonctionnalités comme instrument, etc.).  
**On insistera sur le fait qu’il vaut mieux planifier des
expérimentations ou quasi-expériences a priori (prix A/B test géographique, etc.) que de se reposer sur la
chance d’avoir un bon instrument a posteriori**.  
Néanmoins, savoir appliquer cette méthode est un atout
pour les analystes marketing quantitatifs.